# Imágenes satelitales NASA SpaceApps training [@AESS Unicauca](https://www.facebook.com/aess.unicauca)


<div>
<img src="https://scontent-dub4-1.xx.fbcdn.net/v/t39.30808-6/342913800_189099650681320_3258870437073193619_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=09cbfe&_nc_ohc=aDe-aZgTMjcAX8KgZ2W&_nc_ht=scontent-dub4-1.xx&oh=00_AfAA6dVoH2kfafxIfL8XrC46LB9Xh81IoJiLbdtMrg_uhw&oe=64A8B0AE" width="700" height = "400"/>
</div>


En el siguiente notebook se exploran Google Earth Engine para descargar imagenes satelitales.

Por [Sebastian Cajas Ordóñez](https://sebasmos.github.io/)  

## Outline
1. [Google Earth Engine](#8)

<a name='8'></a>
# Google Earth Engine

## Instalación para uso en máquina virtual

In [ ]:
%%bash 
# Grant permissions to run bash script
chmod u+x ../scripts/gee.sh
# Install GEE
../scripts/gee.sh

## Instalación para Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install earthengine-api 
# This tensorflow version is necessary just for tensorboard support.
# developer version, on future it maybe breaks. 
!pip install tf-nightly-2.0-preview==2.0.0.dev20190606 

!pip install earthengine-api==0.1.175 #earthengine API
# Load the TensorBoard notebook extension
%load_ext tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tf-nightly-2.0-preview==2.0.0.dev20190606 (from versions: none)
ERROR: No matching distribution found for tf-nightly-2.0-preview==2.0.0.dev20190606
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 30.8 MB/s 
     |████████████████████████████████| 57 kB 3.8 MB/s 
     |████████████████████████████████| 4.0 MB 64.0 MB/s 
  Created wheel for earthengine-api: filename=earthengine_api-0.1.175-py3-none-any.whl size=144197 sha256=f071fdad9fa384823e52843d6fb571afb706b46e2918b385fe1845869232076a
  Stored in directory: /root/.cache/pip/wheels/bc/53/2b/52b81cb84c66a5ce1da52c69722b7b4b293224e83841664937
Successfully built earthengine-api
  Attem

In [ ]:
!earthengine authenticate 

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=0c6T7P0fe1Y7-XFqGFn_v1EB7r7H7OYEFjm5u8jMM9o&tc=0Ko8rQiX7jjFj1d7PMNCR7_sv0BD4wnxTEq656mghJs&cc=OzwYv85l-RjM433060h4CJUY-vijiCyRZ50SlXeI324

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJrLnYX08DF0EGNp6wlTEUew0myutyL62OdfYv3hX93Togw57SDT_0Y

Successfully saved authorization token.


In [ ]:
# Earth Engine Python API
import ee
from google.colab import auth
auth.authenticate_user()
ee.Initialize()

import tensorflow as tf
print('Tensorflow version: ' + tf.__version__)

import folium
print('Folium version: ' + folium.__version__)

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

Tensorflow version: 2.8.2
Folium version: 0.12.1.post1


In [ ]:
#@title Utility functions: Display ee.Features and ee.Images using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz



def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer



## Visualize regions

In [114]:
# 2.4.1 Prediction Area
xmin,ymin,xmax,ymax = [-76.658611,
                       2.426429,
                       -76.576244,
                       2.491943]


# Passing a rectangle (prediction area) to Earth Engine
Popayan_data = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])
center = Popayan_data.centroid().getInfo()['coordinates']
center.reverse()
Mapdisplay(center,{'Popayan':Popayan_data.getInfo()}, zoom_start=13)

## Download images

In [ ]:
# Load a landsat image and select three bands.
Popayan = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_123032_20140515')\
            .select(['B4', 'B3', 'B2'])

# Create a geometry representing an export region.
geometry = ee.Geometry.Rectangle([-76.658611,
                       2.426429,
                       -76.576244,
                       2.491943])
center = geometry.centroid().getInfo()['coordinates']
center.reverse()

Mapdisplay(center,{'landsat':Popayan.getMapId()},zoom_start=13)

In [ ]:
import time 
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': Popayan,
    'description': 'imageToDriveExample',
    'folder':'Example_folder',
    'scale': 100,
    'region': geometry.getInfo()['coordinates']
})
task.start()
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).
Polling for task (id: VRKPDOL2PCSAU326KAK6HTAN).


## Image processing operations

In [115]:
# Define the visualization parameters.

image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')
center = [37.5010, -122.1899]
image_viz_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
}

# Define a map centered on San Francisco Bay.
map_l8 = folium.Map(location=[37.5010, -122.1899], zoom_start=11)

# Add the image layer to the map and display it.
map_l8.add_ee_layer(image, image_viz_params, 'false color composite')
display(map_l8)

#### Gradient calculation

In [116]:
kernel = ee.Kernel.circle(radius=1)

# Perform an erosion followed by a dilation, display.
opened = image.focal_min(kernel=kernel, iterations=2).focal_max(kernel=kernel, iterations=2)
# Compute the image gradient in the X and Y directions.
xyGrad = image.gradient();

# Compute the magnitude of the gradient.
gradient = xyGrad.select('x').pow(2).add(xyGrad.select('y').pow(2)).sqrt();

# Compute the direction of the gradient.
direction = xyGrad.select('y').atan2(xyGrad.select('x'));

# Display the results.
tiles = {
    'direction': direction.getMapId({'min': -2, 'max': 2}),
    'gradient' : opened.getMapId({'min': -7, 'max': 7})
}
Mapdisplay(center, tiles, "Stamen Terrain")

#### NDWI Layer

In [117]:
# Load an image.
# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwi_viz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}

# Define a map centered on San Francisco Bay.
map_ndwi = folium.Map(location=center, zoom_start=10)

# Add the image layer to the map and display it.
map_ndwi.add_ee_layer(ndwi, ndwi_viz, 'NDWI')
display(map_ndwi)

#### Convolutions

In [118]:
# Define a boxcar or low-pass kernel
boxcar  = ee.Kernel.square(7, 'pixels', True)

# Smooth the image by convolving with the boxcar kernel
smooth = image.convolve(boxcar)
  
tiles = {
    "input image": image.getMapId({'bands': ['B5', 'B4', 'B3'], 'max': 0.5}),
    "smoothed": smooth.getMapId({'bands': ['B5', 'B4', 'B3'], 'max': 0.5})
}

center = [37.8694, -121.9785]
Mapdisplay(center, tiles, "Stamen Terrain")

In [ ]:
center

[2.459186366404102, -76.61742749999729]

References:

[Google Earth Engine with Python (EEwPython)](https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/10_Export.ipynb#scrollTo=5VhMvaETVkXX).

[Introduction to remote Sensing](https://worldbank.github.io/OpenNightLights/tutorials/mod1_1_introduction_to_remote_sensing.html)

[Spatial thoughts tutorials](https://courses.spatialthoughts.com/end-to-end-gee.html)